In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
DATA_DIR = "Faulty_solar_panel"
BATCH_SIZE = 32
NUM_CLASSES = 6
EPOCHS = 10
LR = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])  
])

In [4]:
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
class_names = full_dataset.classes
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE)

/home/antrskarya/VScode/EAD-assignment/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/antrskarya/VScode/EAD-assignment/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/antrskarya/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [8]:
print("Starting training...\n")
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()

    acc = 100 * correct / len(train_ds)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss:.4f}, Accuracy: {acc:.2f}%")

Starting training...

Epoch 1/10, Loss: 25.6964, Accuracy: 61.86%
Epoch 2/10, Loss: 14.7088, Accuracy: 79.38%
Epoch 3/10, Loss: 14.3148, Accuracy: 81.21%
Epoch 4/10, Loss: 10.6482, Accuracy: 84.32%
Epoch 5/10, Loss: 12.6204, Accuracy: 86.16%
Epoch 6/10, Loss: 10.9744, Accuracy: 84.04%
Epoch 7/10, Loss: 7.9842, Accuracy: 87.71%
Epoch 8/10, Loss: 6.9742, Accuracy: 91.24%
Epoch 9/10, Loss: 7.2918, Accuracy: 91.67%
Epoch 10/10, Loss: 7.9134, Accuracy: 90.40%


In [9]:
print("\nEvaluating on validation set...\n")
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(DEVICE)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=class_names))


Evaluating on validation set...

[[30  0  3  6  1  2]
 [ 0 29  3  9  0  0]
 [ 3  5 29  2  0  1]
 [ 0  0  0 20  0  0]
 [ 4  0  0  3  6  0]
 [ 1  2  2  0  0 16]]
                   precision    recall  f1-score   support

        Bird-drop       0.79      0.71      0.75        42
            Clean       0.81      0.71      0.75        41
            Dusty       0.78      0.72      0.75        40
Electrical-damage       0.50      1.00      0.67        20
  Physical-Damage       0.86      0.46      0.60        13
     Snow-Covered       0.84      0.76      0.80        21

         accuracy                           0.73       177
        macro avg       0.76      0.73      0.72       177
     weighted avg       0.77      0.73      0.74       177



In [10]:
torch.save(model.state_dict(), "resnet18_solar_panel.pth")